In [ ]:
#addprocs()
using Intan

In [ ]:
#In this example, we will consider one 64 channel RHD2164 amp 
#connected to Port A1 of an Intan evaluation board

#First create amplifier array:
myamp=RHD2164("PortA1")

#Initialize evaluation board with appropriate amplifier vector and 
#sampling rate
sr=20000

myrhd=init_board([myamp],sr)


In [ ]:
#1 second sequence for calibration
USB_BUFFER_SIZE = 2400000
setMaxTimeStep(600) #set num samples to acquire
setContinuousRunMode(false)
mytime=zeros(Int32,2)
usbBuffer = ones(Uint8,USB_BUFFER_SIZE)
usbBuffer=convert(SharedArray{Uint8,1},usbBuffer);

In [ ]:
runBoard()

In [ ]:
usbDataRead=true

runBoard()

while (usbDataRead || isRunning())
    #If Fifo contains words, they are read
    @time usbDataRead = readDataBlocks(1,mytime,s,"METHOD_SORTCAL")  
end

flushBoard()

In [ ]:
setMaxTimeStep(20000) #set num samples to acquire

#prepare output file
saveOut="test.jld"
h5open(saveOut, "w") do myfile
    d=d_create(myfile, "time", Int32, ((1,),(-1,)), "chunk", (100,))
    d[1]=1
    for i=1:64
        e=d_create(myfile, string(i), Int64, ((1,),(-1,)),"chunk", (100,))
        e=d_create(myfile, string("n",i), Int64, ((1,),(-1,)),"chunk", (100,))
    end
end

In [ ]:
runBoard()

while (usbDataRead || isRunning())
    usbDataRead = readDataBlocks(1,mytime,s,"METHOD_SORT")
    if (length(mytime) >= 3000)
        queueToFile(mytime,s,saveOut)
    end
    
end

queueToFile(mytime,s,saveOut)

flushBoard()

In [ ]:
#print the output of a set of time stamps to make sure they are not mistaken

h5open(saveOut) do hi
    e=d_open(hi, "time")
    println(e[:])
end